# 0.Libraray Import & Data Load
- 유동인구 데이터 (SKT)
- 카드데이터 (신한카드)
- 유통데이터 (편의점, GS)

- 외부데이터
  - 코로나데이터 (코로나 확진자 수 데이터)
  - 휴일 여부 데이터 (2020, 2019)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 분석 library
import pandas as pd
import numpy as np

# 통계용 library
import scipy
from scipy.stats.stats import pearsonr # 상관계수
import statsmodels.formula.api as smf # 회귀모델용

!pip install pingouin
import pingouin as pg # 편상관분석용

# 기타
from functools import reduce
import datetime
from datetime import datetime, timedelta
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 225kB 2.7MB/s 
  Created wheel for pingouin: filename=pingouin-0.3.8-cp36-none-any.whl size=221688 sha256=67566f675254332c87800cb02e0fa30e27adeac46f5142d302c7bb28b10230e8
  Stored in directory: /root/.cache/pip/wheels/d6/9e/53/f885f73f29cf7c8cac3d8f4b1532bbfef2f5eb543946ac9055
  Created wheel for outdated: filename=outdated-0.2.0-cp36-none-any.whl size=4961 sha256=b8150d961f6bb13e9db0d1b12c6580e47c54cb9390ef4b45cf4b358d97b5313c
  Stored in directory: /root/.cache/pip/wheels/fd/7c/ef/814f514d31197310872b5abf353feb8fef9d67ee658e1e7e39
  Created wheel for littleutils: filename=littleutils-0.2.2-cp36-none-any.whl size=7051 sha256=72a0a554f3bf6976807417173ba394113e95f1d1e065e8094c763c1bf434a6a5
  Stored in directory: /root/.cache/pip/wheels/53/16/9f/ac67d15c40243754fd73f620e1b9b6dedc20492ecc19a2bae1
Successfully built pingouin outdated littleutils


## 유동인구 데이터 (2020, 2019)
- SKT 유동인구 데이터 2020, 2019 사용

In [ ]:
# 2020년 데이터
skt_age_202002 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202002.CSV", sep="|")
skt_age_202003 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202003.CSV", sep="|")
skt_age_202004 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202004.CSV", sep="|")
skt_age_202005 = pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_202005.CSV", sep="|")

# 2019 유동인구 데이터 age
skt_age_201902=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201902.CSV", sep="|")
skt_age_201903=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201903.CSV", sep="|")
skt_age_201904=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201904.CSV", sep="|")
skt_age_201905=pd.read_csv("/content/drive/My Drive/big_contest/skt/4개지역_FLOW_AGE_201905.CSV", sep="|")

# skt 데이터 합침 (2020, 2019)
skt_2020 = pd.concat([skt_age_202002,skt_age_202003,skt_age_202004,skt_age_202005]) # 2020년 데이터
skt_2019 = pd.concat([skt_age_201902, skt_age_201903, skt_age_201904, skt_age_201905]) # 2019년 데이터

print(skt_2020.shape, skt_2019.shape)
skt_2020

(8349, 34) (8280, 34)


,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,MAN_FLOW_POP_CNT_4044,MAN_FLOW_POP_CNT_4549,MAN_FLOW_POP_CNT_5054,MAN_FLOW_POP_CNT_5559,MAN_FLOW_POP_CNT_6064,MAN_FLOW_POP_CNT_6569,MAN_FLOW_POP_CNT_70U,WMAN_FLOW_POP_CNT_0004,WMAN_FLOW_POP_CNT_0509,WMAN_FLOW_POP_CNT_1014,WMAN_FLOW_POP_CNT_1519,WMAN_FLOW_POP_CNT_2024,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
0,202002,20200201,1114052000,소공동,0.0,88.95,376.68,1077.03,3602.41,6660.39,6519.52,5675.54,5063.35,5992.92,6207.66,5997.45,6356.51,4740.35,5766.20,0.0,73.78,433.73,1408.06,4999.85,6961.02,5115.24,4164.47,3927.93,4058.47,3932.39,3650.70,3361.05,2440.61,2665.43
1,202002,20200202,1114052000,소공동,0.0,51.03,271.96,871.27,2824.97,5031.63,4626.14,3975.63,3455.81,4218.84,4473.19,4183.04,4278.93,3182.24,3567.28,0.0,55.15,327.72,1231.75,3807.14,4702.91,3698.04,2922.33,2781.63,2947.01,2632.34,2669.56,2399.90,1704.22,2026.77
2,202002,20200203,1114052000,소공동,0.0,32.47,136.94,826.26,3580.23,8823.11,11803.45,14934.31,14823.99,16430.05,14201.85,10908.99,9628.67,6658.04,7411.35,0.0,35.09,155.99,1166.65,6301.35,11109.59,10754.75,9655.01,9018.58,8786.59,7257.35,5710.55,4935.00,3070.67,3349.99
3,202002,20200204,1114052000,소공동,0.0,27.34,154.75,823.69,3510.75,9361.85,12127.55,15650.18,15484.99,16854.01,14653.71,11550.17,9567.69,6884.62,7319.09,0.0,40.41,193.20,1148.80,6345.14,11771.06,11467.86,9834.22,9323.74,8982.67,7448.09,5938.57,4948.10,3091.30,3494.91
4,202002,20200205,1114052000,소공동,0.0,26.30,177.28,809.21,3579.35,9631.85,12378.43,15876.88,15624.94,16730.33,14462.22,11456.97,9360.22,6709.08,6951.99,0.0,36.44,172.20,1285.73,6587.78,11876.40,11285.83,9972.17,9299.52,8976.55,7183.79,5719.89,4822.39,2960.73,3204.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,202005,20200527,2726069000,고산3동,0.0,70.07,519.05,956.45,1205.44,1507.57,1500.51,2014.87,2295.21,2947.78,3020.17,2653.97,2217.68,1561.13,1492.80,0.0,114.34,552.87,842.82,1264.78,1201.14,1106.57,1728.34,2209.31,2808.54,2721.54,2070.62,1613.50,1086.34,1143.37
2135,202005,20200528,2726069000,고산3동,0.0,99.04,749.36,1085.68,1639.82,1777.32,1776.02,2137.50,2582.56,3415.30,3458.23,3019.66,2550.45,1845.30,1780.62,0.0,146.62,733.55,1117.85,1597.98,1377.94,1258.02,1847.61,2677.19,3342.68,3036.71,2389.70,1848.92,1291.19,1515.36
2136,202005,20200529,2726069000,고산3동,0.0,155.70,781.70,1235.94,1706.97,1982.74,1924.60,2500.02,2810.11,3625.34,3718.35,3212.18,2852.53,1755.48,1811.35,0.0,111.30,801.94,1167.51,1709.27,1583.58,1487.06,2149.30,2916.87,3643.31,3395.24,2800.02,2043.25,1356.15,1455.89
2137,202005,20200530,2726069000,고산3동,0.0,119.89,903.85,1236.60,1722.01,2142.41,2106.49,2215.86,2649.30,3713.62,3854.18,3138.66,2506.90,1683.99,1666.55,0.0,152.58,727.46,1229.17,1627.58,1644.73,1420.31,1812.27,2640.71,3362.25,2909.34,2353.82,1812.56,1242.56,1372.38


In [ ]:
# skt 유동인구 데이터

# 2020 유동인구 데이터 연령 총합 변수 만들기
skt_2020['sum_20'] = skt_2020.iloc[:,4:].sum(axis=1)
skt_2020 = skt_2020[['STD_YMD','HDONG_NM','sum_20']]
skt_2020.columns = ['STD_DD','HDONG_NM','sum_20']
skt_2020 = skt_2020.reset_index(drop=True)

# 2019 유동인구 데이터 연령 총합 변수 만들기
skt_2019["sum_19"] = skt_2019.iloc[:,4:].sum(axis=1)
skt_2019 = skt_2019[["STD_YMD","HDONG_NM","sum_19"]]
skt_2019.columns = ['STD_DD','HDONG_NM','sum_19']
skt_2019 = skt_2019.reset_index(drop=True)

print(skt_2020.shape, skt_2019.shape)
skt_2019

(8349, 3) (8280, 3)


,STD_DD,HDONG_NM,sum_19
0,20190201,소공동,242230.68
1,20190202,소공동,133411.99
2,20190203,소공동,90124.93
3,20190204,소공동,83378.55
4,20190205,소공동,111009.47
...,...,...,...
8275,20190527,고산3동,47011.48
8276,20190528,고산3동,49344.74
8277,20190529,고산3동,50345.41
8278,20190530,고산3동,50148.22


## 카드데이터 (2020, 2019)
- 신한카드 데이터 (외국인 + 내국인)
- 신한카드 행정동 데이터
- 카드 데이터 + 행정동 데이터 Merge
- 2020, 2019 데이터 분리
- 카드데이터 업종별 대분류 리스트 변수 생성

In [ ]:
# 신한 카드 데이터 (내국인, 외국인)
card_re = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_RESIDENT.txt', sep = "\t", engine='python', encoding = "cp949")
card_fo = pd.read_csv('/content/drive/My Drive/big_contest/shinhan/CARD_SPENDING_FOREIGNER.txt', sep = "\t", engine='python', encoding = "cp949")

# 신한 카드 행정동 데이터
dong = pd.read_excel('/content/drive/My Drive/big_contest/shinhan/02_혁신아이디어분야_카드소비데이터(신한카드)_데이터정의서.xlsx', sheet_name='(참고) 행정동코드',index_col=0, header=None)

print(card_re.shape, card_fo.shape, dong.shape)

(2537753, 8) (238829, 7) (71, 4)


In [ ]:
# 카드 행정동 데이터 전처리
dong.columns = ['GU_CD','DONG_CD','GU_NM','HDONG_NM']
dong.reset_index(inplace=True, drop=True)
dong = dong.loc[2:]
dong = dong.reset_index(drop=True)

# 컬럼 type int 형으로 변경
dong['GU_CD'] = dong['GU_CD'].astype(int)
dong['DONG_CD'] = dong['DONG_CD'].astype(int)

# 카드 데이터 (내국인 + 외국인) concat
card = pd.concat([card_fo,card_re])

# 행정동 이름과 카드 데이터 merge
card = pd.merge(card, dong, on=['GU_CD','DONG_CD'])

print(card.shape)
card

(2776582, 11)


,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,COUNTRY_NM,USE_CNT,USE_AMT,SEX_CD,AGE_CD,GU_NM,HDONG_NM
0,20190204,260,510,81,대만,2,30.0,NaN,NaN,대구 수성구,범어1동
1,20190213,260,510,40,대만,2,32.0,NaN,NaN,대구 수성구,범어1동
2,20190416,260,510,40,대만,2,16.0,NaN,NaN,대구 수성구,범어1동
3,20190419,260,510,42,대만,3,610.0,NaN,NaN,대구 수성구,범어1동
4,20190308,260,510,10,대만,7,629.0,NaN,NaN,대구 수성구,범어1동
...,...,...,...,...,...,...,...,...,...,...,...
2776577,20200531,350,710,81,NaN,47,570.0,M,45.0,서울 노원구,상계9동
2776578,20200531,350,710,81,NaN,33,444.0,M,50.0,서울 노원구,상계9동
2776579,20200531,350,710,81,NaN,28,403.0,M,55.0,서울 노원구,상계9동
2776580,20200531,350,710,81,NaN,33,384.0,M,60.0,서울 노원구,상계9동


In [ ]:
# 일별, 구별 ,동별 USE_AMT, USE_AMT 합
card_group = card.groupby(["STD_DD",'GU_NM','HDONG_NM'], as_index=False)['USE_CNT','USE_AMT'].agg("sum")

# 2020, 2019 카드 데이터 분리
card_2020 = card_group[card_group.STD_DD>=20200101] # 2020년 카드 데이터
card_2019 = card_group[card_group.STD_DD<20200101] # 2019년 카드 데이터

print(card_2020.shape, card_2019.shape)

(8349, 5) (8280, 5)


In [ ]:
# 카드 대분류 list dictionary
type_list = {'10':'숙박(10)', '20':'레저용품(20)', '21':'레저업소(21)', 
            '22':'문화취미(22)', '30':'가구(30)' , '31':'전기(31)' ,
            '32':'주방용구(32)', '33':'연료판매(33)' , '34':'광학제품(34)',
            '35':'가전(35)', '40':'유통업(40)', '42':'의복(42)', 
            '43':'직물(43)', '44':'신변잡화(44)', '50':'서적문구(50)',
            '52':'사무통신(52)', '60':'자동차판매(60)','62':'자동차정비(62)',
            '70':'의료기관(70)', '71':'보건위생(71)', '80':'요식업소(80)',
            '81':'음료식품(81)', '92':'수리서비스(92)'}

# 업종 list
MCT_CAT_CD_list = card['MCT_CAT_CD'].unique()
MCT_CAT_CD_list = sorted(MCT_CAT_CD_list)
MCT_CAT_CD_list = list(map(str, MCT_CAT_CD_list))
print(len(MCT_CAT_CD_list), MCT_CAT_CD_list)

23 ['10', '20', '21', '22', '30', '31', '32', '33', '34', '35', '40', '42', '43', '44', '50', '52', '60', '62', '70', '71', '80', '81', '92']


## 편의점(유통, GS) 데이터 (2020,2019)
- GS 제공 편의점 데이터
- 구조방정식, 매개효과용 데이터 변수 : gs
- 회귀용 데이터 변수명 : gs_row -> 2020, 2019 분리해서 사용

In [ ]:
# GS 데이터 가져오기
gs = pd.read_excel('/content/drive/My Drive/big_contest/GS/유통(GS리테일).xlsx', sheet_name='종합테이블(TB_DD_STR_PURCH_CLS)')
print(gs.shape)
gs

(14460, 13)


,OPER_DT,PVN,BOR,ADMD,AMT_IND,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P
0,20190201,대구광역시,수성구,고산1동,1.064336,0.450848,0.172221,0.295458,0.023263,0.044679,0.005075,0.004820,0.003637
1,20190201,대구광역시,수성구,고산2동,1.076418,0.457937,0.196274,0.295564,0.005521,0.034056,0.000844,0.008893,0.000913
2,20190201,대구광역시,수성구,고산3동,0.856071,0.359978,0.196602,0.376762,0.013408,0.029606,0.009117,0.008831,0.005697
3,20190201,대구광역시,수성구,두산동,0.724868,0.281969,0.175207,0.363202,0.024235,0.134191,0.000000,0.021196,0.000000
4,20190201,대구광역시,수성구,만촌1동,1.155728,0.400983,0.200454,0.310531,0.038868,0.035427,0.005284,0.005734,0.002719
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14455,20200531,서울특별시,중구,중림동,1.595038,0.314217,0.221087,0.344307,0.026204,0.070094,0.006347,0.013541,0.004203
14456,20200531,서울특별시,중구,청구동,1.389184,0.276333,0.221216,0.363681,0.053907,0.076963,0.001592,0.000353,0.005955
14457,20200531,서울특별시,중구,필동,0.714265,0.303338,0.187615,0.370371,0.038908,0.079408,0.007826,0.008298,0.004236
14458,20200531,서울특별시,중구,황학동,1.134190,0.264708,0.257387,0.367001,0.029140,0.070770,0.001060,0.003646,0.006287


In [ ]:
# 차후 업종별 상관계수 및 회귀 모델용 데이터 변수
gs_row = gs.copy()

# GS 데이터 필요 column 줄이기 (구조방정식, 매개효과 용 변수)
gs.columns=['STD_DD', 'HGU_NM', 'CTGG_NM', 'HDONG_NM', 'AMT_IND', 'LCLS_10_P', 'LCLS_20_P',
            'LCLS_30_P', 'LCLS_40_P', 'LCLS_50_P', 'LCLS_60_P', 'LCLS_70_P', 'LCLS_80_P']
gs = gs[['STD_DD', 'HGU_NM', 'HDONG_NM', 'AMT_IND']]
gs

,STD_DD,HGU_NM,HDONG_NM,AMT_IND
0,20190201,대구광역시,고산1동,1.064336
1,20190201,대구광역시,고산2동,1.076418
2,20190201,대구광역시,고산3동,0.856071
3,20190201,대구광역시,두산동,0.724868
4,20190201,대구광역시,만촌1동,1.155728
...,...,...,...,...
14455,20200531,서울특별시,중림동,1.595038
14456,20200531,서울특별시,청구동,1.389184
14457,20200531,서울특별시,필동,0.714265
14458,20200531,서울특별시,황학동,1.134190


In [ ]:
# 유통 GS 품목별 대분류 list dictionary
circulation_list = {'mul_LCLS_10_P':'식사',
                    'mul_LCLS_20_P':'간식',
                    'mul_LCLS_30_P':'마실거리',
                    'mul_LCLS_40_P':'홈&리빙',
                    'mul_LCLS_50_P':'헬스&뷰티',
                    'mul_LCLS_60_P':'취미&여가활동_비중',
                    'mul_LCLS_70_P':'사회활동_비중',
                    'mul_LCLS_80_P':'임신/육아_비중',
                    'mul_LCLS_90_P':'기호품_비중'}

# 유통 GS 품목별 대분류 list dictionary
circulation_list_reverse = {'식사':               'mul_LCLS_10_P',
                            '간식':               'mul_LCLS_20_P',
                            '마실거리' :          'mul_LCLS_30_P',
                            '홈&리빙':            'mul_LCLS_40_P',
                            '헬스&뷰티':          'mul_LCLS_50_P',
                            '취미&여가활동_비중': 'mul_LCLS_60_P',
                            '사회활동_비중':      'mul_LCLS_70_P',
                            '임신/육아_비중':     'mul_LCLS_80_P',
                            '기호품_비중':        'mul_LCLS_90_P'}

## 코로나 데이터 (2020)
- 출처 : https://github.com/jooeungen/coronaboard_kr // CoronaBoard_kr 깃허브 
- 출처 : http://ncov.mohw.go.kr/ // corona board에서 참조한 질병관리본부 링크
- <b>CoronaBoard data - Available Commercial & Non-commercial</b>
  - kr_daily.csv (코로나 일일 확진자 수 - 누적) (원본)
  - kr_regional_data.csv (코로나 지역별  일일 확진자 수 - 누적 )(원본)

- <b>사용데이터 : 코로나 일일 확진자 수 데이터 (전국, 서울, 대구)</b>
  - <b>위 출처 데이터를 1차 가공(차분)한 데이터 (누적 -> 일일) </b>
  - 기간 : 20200201 ~ 20200601 데이터 사용
  - covid_all.csv : 전국 코로나 일일 확진자 수
  - corona_seoul.csv : 서울지역 코로나 일일 확진자 수
  - corona_daegu.csv : 대구지역 코로나 일일 확진자 수
  - 코로나 데이터는 하루씩 당겨서 사용
  

In [ ]:
# 코로나 전국 확진자 데이터
covid = pd.read_csv("/content/drive/My Drive/big_contest/corona/covid_all.csv", index_col=0, encoding="cp949")

# 서울, 대구 각각의 지역 확진자 데이터
covid_seoul = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_seoul.csv", index_col=0, encoding="cp949")
covid_daegu = pd.read_csv("/content/drive/My Drive/big_contest/corona/corona_daegu.csv", index_col=0, encoding="cp949")

# 전국 코로나 데이터
covid = covid.reset_index()
covid.columns = ['STD_DD','confirmed']
covid

,STD_DD,confirmed
0,20200201,4
1,20200202,1
2,20200203,3
3,20200204,0
4,20200205,1
...,...,...
179,20200729,28
180,20200730,48
181,20200731,18
182,20200801,36


In [ ]:
# corona 지역 데이터 (서울, 대구)
covid_seoul = covid_seoul[['date','confirmed_diff']]
covid_seoul.columns=['STD_DD','seoul_confirmed']
covid_daegu = covid_daegu[['date','confirmed_diff']]
covid_daegu.columns=['STD_DD','daegu_confirmed']

# 서울, 대구 코로나 데이터
print(covid_seoul.shape, covid_daegu.shape)
covid_seoul.head(3)

(121, 2) (121, 2)


,STD_DD,seoul_confirmed
1,20200201,0
2,20200202,1
3,20200203,0


## 휴일여부 데이터 (2019, 2020)
- 2019년, 2020년 휴일 여부 데이터
- 직접 데이터 수작업 작성
- 명절, 연휴, 대체휴일 등 반영

In [ ]:
# 2019, 2020 휴일 여부 데이터
holiday = pd.read_csv("/content/drive/My Drive/big_contest/holiday/holiday.csv")

print(holiday.shape)
holiday

(241, 3)


,date,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


In [ ]:
# 컬럼명 변경, type 변경
holiday.columns = ['STD_DD','day','weekend']

holiday['STD_DD'] = holiday['STD_DD'].astype(str)
holiday['STD_DD'] = holiday['STD_DD'].str.replace("-","")
holiday['STD_DD'] = holiday['STD_DD'].astype(int)
holiday

,STD_DD,day,weekend
0,20190201,4,0
1,20190202,5,1
2,20190203,6,1
3,20190204,0,1
4,20190205,1,1
...,...,...,...
236,20200527,2,0
237,20200528,3,0
238,20200529,4,0
239,20200530,5,1


## 서울 대구 행정동 리스트
- 서울 중구/노원구 행정동 리스트
- 대구 중구/수성구 행정동 리스트

In [ ]:
# 분석의 편의를 위해 서울시 중구, 서울시 노원구, 대구시 중구, 대구시 수성구 동별로 리스트 생성
seoul_jung_list = ['소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동', '신당동', '다산동','약수동', '청구동', '신당5동', '동화동', '황학동', '중림동']
seoul_nowon_list = ['월계1동', '월계2동', '월계3동','공릉1동', '공릉2동', '하계1동', '하계2동', '중계본동', '중계1동', '중계4동', '중계2.3동','상계1동', '상계2동', '상계3.4동', '상계5동', '상계6.7동', '상계8동', '상계9동','상계10동']
daegu_jung_list = ['동인동', '삼덕동', '성내1동', '성내2동', '성내3동', '대신동', '남산1동', '남산2동', '남산3동', '남산4동', '대봉1동', '대봉2동']
daegu_suseong_list = ['범어1동', '범어2동', '범어3동', '범어4동', '만촌1동', '만촌2동', '만촌3동', '수성1가동', '수성2.3가동', '수성4가동', '황금1동', 
                      '황금2동', '중동', '상동', '파동', '두산동', '지산1동', '지산2동', '범물1동', '범물2동', '고산1동', '고산2동', '고산3동']

# 3.매개효과 & 부트스트래핑 검정
- 신규 코로나 확진자(𝑿)와 오프라인 매출액(𝐘)의 관계에서 2020 유동인구(𝐌)를 매개변인으로 한 매개모델을 가정

- 종속변수를 2020 유동인구(𝑴), 독립변수를 신규 코로나 확진자(𝑿)로 한 단일회귀 (𝐌= 𝜷_𝟎+𝒂∙𝑿) 의 비표준화 회귀계수(𝒂)를 구함

- 종속변수를 2020 카드/GS 매출액(𝐘), 독립변수를 신규 코로나 확진자(𝑿), 2020 유동인구(𝑴)인 다중 회귀(𝐘= 𝜷_𝟎+𝒃∙𝑿+𝒄∙𝑴)에서의 비표준화 회귀계수(𝒃, 𝒄) 를 구함

## 부트스트래핑 검정 소스코드
- human_learning_3_1_부트스트랩핑카드평일.ipynb
- human_learning_3_2_부트스트랩핑카드휴일.ipynb
- human_learning_3_3_부트스트랩핑편의점평일.ipynb
- human_learning_3_4_부트스트랩핑편의점휴일.ipynb

- 위 4개의 ipynb에서 얻은 파일들
  - 카드평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 카드휴일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점휴일부트스트래핑_신뢰95_비표준화_10000.csv

## 순수 매개효과 조건 1,2,3
- human_learning_3_0_부트스트랩핑카드평일.ipynb 에서 위 4개의 ipynb파일 결과를 종합 받고 조건1,2,3 에 대해서 진행 -> 최종 순수매개효과 지역 추출

### 조건1
- 간접효과 𝒂⦁𝒄는 통계적 유의성을 위해 Bootstrapping 검정을 진행하며, 직접효과 𝒃는 p-value 검정을 통해 유의 여부를 판단한다 

- 회귀계수 𝒂, 𝒄 각각은 t분포를 가정하고 통계적 유의성을 검정
- 𝒂⦁𝒄 는 회귀계수 𝒂 와 𝒄 의 Cross Product 이기 때문에 분포가 불분명
- 따라서 Bootstrapping 기법을 사용해 𝒂⦁𝒄 의 표본 분포를 추정하고 신뢰구간으로 통계적 유의성을 검증

- 따라서 해당 데이터를 가지고 10000번의 30% random sampling을 통해 모집단을 대표할 수 있는 분포를 생성

- 그 후 95% 신뢰구간에서 0이 포함된 매개효과 통계량을 통계적으로 유의하지 않다고 판단

### 조건2 
- 직접효과(b)는 유의하지 않고, 간접효과(ac)만 유의한 경우 분석 대상에 포함
- 직접효과 (b)가 유의시 해당 지역은 기각

### 조건3
- a가 음수 c가 양수인경우만 선택 
- 신규 코로나 확진자 증가로인해 유동인구가 감소하였고, 유동인구 감소로 인해 매출도 같이 감소한 경우

### 최종 결과 순수 매개효과, Bootstraping, 조건1,2,3 결과 
  - boot_direct_mediation_result.csv



## 카드/편의점 데이터 부트스트래핑 동 리스트 전달
- 부트스트래핑 작업이 시간이 오래걸려서 (약 4시간 ~ 7시간 소요) 따로 ipynb 파일 작성
- 부트스트래핑 돌릴 동 리스트 .csv => mediation_result.csv
- card_weekday_list : 카드 평일 동리스트
- card_weekend_list : 카드 휴일 동리스트
- gs_weekday_list : 편의점 평일 동리스트
- gs_weekend_list : 편의점 휴일 동리스트

- 각 부트스트래핑 ipynb로 전달 후 결과 받은 파일에서 기각 결과 및 직접효과 검정 진행

- 최종 통과한 행정동 리스트 만듬

In [ ]:
# 카드 + 유동인구 + 코로나 + 휴일여부
all=pd.merge(covid, card_group, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on ='STD_DD')
all=all[['STD_DD', 'confirmed','USE_CNT', 'sum_20', 'GU_NM', 'HDONG_NM','day','weekend']]

card_weekday_list = all['HDONG_NM'].unique()
card_weekend_list = all['HDONG_NM'].unique()

all

,STD_DD,confirmed,USE_CNT,sum_20,GU_NM,HDONG_NM,day,weekend
0,20200201,4,14352,132881.50,대구 수성구,고산1동,5,1
1,20200201,4,10433,232618.75,대구 수성구,고산2동,5,1
2,20200201,4,10618,49579.14,대구 수성구,고산3동,5,1
3,20200201,4,17469,96455.73,대구 수성구,두산동,5,1
4,20200201,4,10964,104249.83,대구 수성구,만촌1동,5,1
...,...,...,...,...,...,...,...,...
8344,20200531,39,7713,21982.16,서울 중구,중림동,6,1
8345,20200531,39,4061,15985.86,서울 중구,청구동,6,1
8346,20200531,39,10271,53756.05,서울 중구,필동,6,1
8347,20200531,39,12780,28761.22,서울 중구,황학동,6,1


In [ ]:
# 편의점 + 코로나 + 휴일 + 유동인구
# skt19_20 = pd.merge(skt_2020, skt_2019, on=['STD_DD','HDONG_NM'])
all=pd.merge(covid, gs, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on='STD_DD')
all=all[['STD_DD', 'confirmed','AMT_IND', 'sum_20', 'HGU_NM', 'HDONG_NM','day','weekend']]

gs_weekday_list = all['HDONG_NM'].unique()
gs_weekend_list = all['HDONG_NM'].unique()

all

,STD_DD,confirmed,AMT_IND,sum_20,HGU_NM,HDONG_NM,day,weekend
0,20200201,4,0.854239,132881.50,대구광역시,고산1동,5,1
1,20200201,4,1.080543,232618.75,대구광역시,고산2동,5,1
2,20200201,4,0.759088,49579.14,대구광역시,고산3동,5,1
3,20200201,4,0.911892,96455.73,대구광역시,두산동,5,1
4,20200201,4,1.124104,104249.83,대구광역시,만촌1동,5,1
...,...,...,...,...,...,...,...,...
7255,20200531,39,1.595038,21982.16,서울특별시,중림동,6,1
7256,20200531,39,1.389184,15985.86,서울특별시,청구동,6,1
7257,20200531,39,0.714265,53756.05,서울특별시,필동,6,1
7258,20200531,39,1.134190,28761.22,서울특별시,황학동,6,1


In [ ]:
# 매개 효과 동리스트 결과 저장
mediation_list = [','.join(card_weekday_list),
                    ','.join(card_weekend_list),
                    ','.join(gs_weekday_list),
                    ','.join(gs_weekend_list)]

mediation_result = pd.DataFrame(columns=['mediation_list'], data=mediation_list)
mediation_result.to_csv('/content/drive/My Drive/big_contest/bootstraping/mediation_result.csv', index=False, encoding='cp949')

print('매개효과 결과 csv 저장 완료')

매개효과 결과 csv 저장 완료


## 부트스트래핑 진행 
### ----------------------------------------------------------------------------
- @@@@@@@@@부트스트래핑 진행 @@@@@@@@@@@
- 4개의 ipynb 에서 진행 각각 colab 기준 7시간
  - human_learning_3_1_부트스트랩핑카드평일
  - human_learning_3_2_부트스트랩핑카드휴일
  - human_learning_3_3_부트스트랩핑편의점평일
  - human_learning_3_4_부트스트랩핑편의점휴일

- 위 4개의 ipynb에서 얻은 파일들
  - 카드평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 카드휴일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점평일부트스트래핑_신뢰95_비표준화_10000.csv
  - 편의점휴일부트스트래핑_신뢰95_비표준화_10000.csv

- @@@@@ 순수 매개효과 보기위한 조건1,2,3 진행 @@@@@
### 이후진행
- 아래 진행은 부트스트래핑 진행완료후 조건1,2,3 제거되는 과정 실행
----------------------------------------------------------------------------


## 조건1 - 부트스트래핑 기각 결과 - 카드, 편의점

In [ ]:
# 카드 부트스트래핑 기각 결과 - 카드
card_boot_weekday = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/카드평일부트스트래핑_신뢰95_비표준화_10000.csv', encoding='cp949')
card_boot_weekend = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/카드휴일부트스트래핑_신뢰95_비표준화_10000.csv', encoding='cp949')
card_boot_weekday.columns=['HDONG_NM','하한','상한','mean']
card_boot_weekend.columns=['HDONG_NM','하한','상한','mean']

card_boot_weekday['ac'] = card_boot_weekday['하한']*card_boot_weekday['상한'] # 사이에 0이 있는거 찾기 위함 곱해서 음수면 사이에 0있음
card_boot_weekend['ac'] = card_boot_weekend['하한']*card_boot_weekend['상한'] # 사이에 0이 있는거 찾기 위함 곱해서 음수면 사이에 0있음

print('평일 0 포함\n', card_boot_weekday[card_boot_weekday['ac']<=0].HDONG_NM,'\n')
print('휴일 0 포함\n', card_boot_weekend[card_boot_weekend['ac']<=0].HDONG_NM)

평일 0 포함
 15    황금1동
45     약수동
Name: HDONG_NM, dtype: object 

휴일 0 포함
 19    남산3동
41     동화동
Name: HDONG_NM, dtype: object


In [ ]:
# 편의점 부트스트래핑 기각 결과
gs_boot_weekday = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/편의점평일부트스트래핑_신뢰95_비표준화_10000.csv', encoding='cp949')
gs_boot_weekend = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/편의점휴일부트스트래핑_신뢰95_비표준화_10000.csv', encoding='cp949')
gs_boot_weekday['ac'] = gs_boot_weekday['하한']*gs_boot_weekday['상한'] # 사이에 0이 있는거 찾기 위함 곱해서 음수면 사이에 0있음
gs_boot_weekend['ac'] = gs_boot_weekend['하한']*gs_boot_weekend['상한'] # 사이에 0이 있는거 찾기 위함 곱해서 음수면 사이에 0있음

print('평일 0 포함', gs_boot_weekday[gs_boot_weekday['ac']<=0].HDONG_NM,'\n')
print('휴일 0 포함', gs_boot_weekend[gs_boot_weekend['ac']<=0].HDONG_NM)

평일 0 포함 Series([], Name: HDONG_NM, dtype: object) 

휴일 0 포함 Series([], Name: HDONG_NM, dtype: object)


## 조건2 - 직접효과(b) 검정 (카드, 편의점)

In [ ]:
# 카드 + 유동인구 + 코로나 + 휴일여부
from tqdm.notebook import tqdm

all=pd.merge(covid, card_group, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on ='STD_DD')
all=all[['STD_DD', 'confirmed','USE_AMT', 'sum_20', 'GU_NM', 'HDONG_NM','day','weekend']]

#평일 카드 b 검정
p_value_week=[]
p_value_week=pd.DataFrame(p_value_week)
for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==0)][['confirmed','USE_AMT','sum_20']]   #휴일은 all.weekend==1
  model = smf.ols(formula = 'USE_AMT ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  p_value_week.loc[i,"p-value"]=df.loc['confirmed','P>|t|']

#휴일 카드 b 검정
p_value_holi=[]
p_value_holi=pd.DataFrame(p_value_holi)
for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==1)][['confirmed','USE_AMT','sum_20']]   #휴일은 all.weekend==1
  model = smf.ols(formula = 'USE_AMT ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  p_value_holi.loc[i,"p-value"]=df.loc['confirmed','P>|t|']

card_direct_mediation_weekday = list(p_value_week[p_value_week['p-value']<0.05].index)
card_direct_mediation_weekend = list(p_value_holi[p_value_holi['p-value']<0.05].index)

print('카드 평일 b 기각', card_direct_mediation_weekday, '\n', len(card_direct_mediation_weekday))
print('카드 휴일 b 기각', card_direct_mediation_weekend, '\n', len(card_direct_mediation_weekend))


카드 평일 b 기각 ['고산1동', '고산2동', '고산3동', '범물2동', '중동', '지산1동', '동인동', '성내3동', '상계2동', '상계6.7동', '상계8동', '월계3동', '중계본동', '광희동', '다산동', '신당동', '필동'] 
 17
카드 휴일 b 기각 ['범어3동', '수성1가동', '수성2.3가동', '수성4가동', '지산1동', '지산2동', '삼덕동', '성내1동', '공릉1동', '상계9동', '월계1동', '월계3동', '중계1동', '하계2동', '광희동', '청구동', '황학동', '회현동'] 
 18


In [ ]:
# 편의점 + 코로나 + 휴일 + 유동인구
all=pd.merge(covid, gs, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on='STD_DD')
all=all[['STD_DD', 'confirmed','AMT_IND', 'sum_20', 'HGU_NM', 'HDONG_NM','day','weekend']]

from tqdm.notebook import tqdm

#평일 편의점 b 검정
p_value_week=[]
p_value_week=pd.DataFrame(p_value_week)
for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==0)][['confirmed','AMT_IND','sum_20']]   #휴일은 all.weekend==1
  model = smf.ols(formula = 'AMT_IND ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  p_value_week.loc[i,"p-value"]=df.loc['confirmed','P>|t|']

#휴일 편의점 b 검정
p_value_holi=[]
p_value_holi=pd.DataFrame(p_value_holi)
for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==1)][['confirmed','AMT_IND','sum_20']]   #휴일은 all.weekend==1
  model = smf.ols(formula = 'AMT_IND ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  p_value_holi.loc[i,"p-value"]=df.loc['confirmed','P>|t|']

gs_direct_mediation_weekday = list(p_value_week[p_value_week['p-value']<0.05].index)
gs_direct_mediation_weekend = list(p_value_holi[p_value_holi['p-value']<0.05].index)

print('편의점 평일 b 기각', gs_direct_mediation_weekday, '\n', len(gs_direct_mediation_weekday))
print('편의점 휴일 b 기각', gs_direct_mediation_weekend, '\n', len(gs_direct_mediation_weekend))


편의점 평일 b 기각 ['만촌1동', '범물2동', '수성2.3가동', '지산1동', '지산2동', '성내2동', '월계2동', '중계2.3동', '중계4동', '중계본동', '하계2동', '광희동', '다산동', '동화동', '명동'] 
 15
편의점 휴일 b 기각 ['범물2동', '지산1동', '지산2동', '황금2동', '중계2.3동', '다산동'] 
 6


## 조건3 - ac 부호 판정 및 제거
- 

In [ ]:
# 조건3 - 카드 a<0 | c>0 부호 확인
direction=[]
direction=pd.DataFrame(direction)
all=pd.merge(covid, card_group, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on ='STD_DD')
all=all[['STD_DD', 'confirmed','USE_AMT', 'sum_20', 'GU_NM', 'HDONG_NM','day','weekend']]

for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==0)][['confirmed','USE_AMT','sum_20']]
  #유동인구~확진자
  model = smf.ols(formula = 'sum_20 ~ confirmed', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'a']=df.loc['confirmed','coef']
  
  #USE_AMT~유동인구+확진자 통제
  model = smf.ols(formula = 'USE_AMT ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'c']=df.loc['sum_20','coef']

# 평일 a플러스 c 마이너스가 아닌 리스트
card_weekday_plus_minus = direction[(direction.a<0) & (direction.c>0)].index
card_weekday_plus_minus = direction[direction.index.isin(set(all.HDONG_NM.unique())-set(card_weekday_plus_minus))]
print(card_weekday_plus_minus.index, len(card_weekday_plus_minus.index))


Index(['범물2동', '지산2동', '파동', '상계3.4동', '상계8동', '상계9동', '중계본동', '다산동', '동화동',
       '황학동'],
      dtype='object') 10


In [ ]:
# 조건3 - 카드 a<0 | c>0 부호 확인
direction=[]
direction=pd.DataFrame(direction)
all=pd.merge(covid, card_group, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on ='STD_DD')
all=all[['STD_DD', 'confirmed','USE_AMT', 'sum_20', 'GU_NM', 'HDONG_NM','day','weekend']]

for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==1)][['confirmed','USE_AMT','sum_20']]

  #유동인구~확진자
  model = smf.ols(formula = 'sum_20 ~ confirmed', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'a']=df.loc['confirmed','coef']

  #USE_AMT~유동인구+확진자 통제
  model = smf.ols(formula = 'USE_AMT ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'c']=df.loc['sum_20','coef']


#휴일 a플러스 c마이너스가 아닌 리스트
card_weekend_plus_minus = direction[(direction.a<0) & (direction.c>0)].index
card_weekend_plus_minus = direction[direction.index.isin(set(all.HDONG_NM.unique())-set(card_weekend_plus_minus))]
print(card_weekend_plus_minus.index, len(card_weekend_plus_minus.index))


Index(['범물2동', '지산2동', '남산3동', '상계3.4동', '상계8동', '다산동', '동화동'], dtype='object') 7


In [ ]:
# 조건3 - 편의점 a<0 | c>0 부호 확인
direction=[]
direction=pd.DataFrame(direction)

# 편의점 + 코로나 + 평일 + 유동인구
all=pd.merge(covid, gs, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on='STD_DD')
all=all[['STD_DD', 'confirmed','AMT_IND', 'sum_20', 'HGU_NM', 'HDONG_NM','day','weekend']]

for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==0)][['confirmed','AMT_IND','sum_20']]
  #유동인구~확진자
  model = smf.ols(formula = 'sum_20 ~ confirmed', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'a']=df.loc['confirmed','coef']

  #AMT_IND~유동인구+확진자 통제
  model = smf.ols(formula = 'AMT_IND ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'c']=df.loc['sum_20','coef']

# 평일 a플러스 c 마이너스가 아닌 리스트
gs_weekday_plus_minus = direction[(direction.a<0) & (direction.c>0)].index
gs_weekday_plus_minus = direction[direction.index.isin(set(all.HDONG_NM.unique())-set(gs_weekday_plus_minus))]
print(gs_weekday_plus_minus.index, len(gs_weekday_plus_minus.index))


Index(['범물2동', '지산2동', '월계2동', '다산동', '동화동'], dtype='object') 5


In [ ]:
# 조건3 - 편의점 a<0 | c>0 부호 확인
direction=[]
direction=pd.DataFrame(direction)

# 편의점 + 코로나 + 휴일 + 유동인구
all=pd.merge(covid, gs, on='STD_DD')
all=pd.merge(all, skt_2020, on=['STD_DD','HDONG_NM'])
all=pd.merge(all, holiday, on='STD_DD')
all=all[['STD_DD', 'confirmed','AMT_IND', 'sum_20', 'HGU_NM', 'HDONG_NM','day','weekend']]

for i in tqdm(all.HDONG_NM.unique()):
  a=all[(all.HDONG_NM==i) & (all.weekend==1)][['confirmed','AMT_IND','sum_20']]

  #유동인구~확진자
  model = smf.ols(formula = 'sum_20 ~ confirmed', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'a']=df.loc['confirmed','coef']

  #AMT_IND~유동인구+확진자 통제
  model = smf.ols(formula = 'AMT_IND ~ confirmed +sum_20', data = a)
  result = model.fit()
  df = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]
  direction.loc[i,'c']=df.loc['sum_20','coef']

# 휴일 a플러스 c 마이너스가 아닌 리스트
gs_weekend_plus_minus = direction[(direction.a<0) & (direction.c>0)].index
gs_weekend_plus_minus = direction[direction.index.isin(set(all.HDONG_NM.unique())-set(gs_weekend_plus_minus))]
print(gs_weekend_plus_minus.index, len(gs_weekend_plus_minus.index))


Index(['범물2동', '지산2동', '상계3.4동', '다산동', '동화동'], dtype='object') 5


## 순수 매개효과 행정동 리스트

In [ ]:
# 조건 검증 전 행정동 리스트
mediation_result = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/mediation_result.csv', encoding='cp949')

# 조건 검증 전 리스트
card_weekday_list = mediation_result.iloc[0, ].values[0].split(',')
card_weekend_list = mediation_result.iloc[1, ].values[0].split(',')
gs_weekday_list = mediation_result.iloc[2, ].values[0].split(',')
gs_weekend_list = mediation_result.iloc[3, ].values[0].split(',')

print('카드 평일 매개 존재 검증 리스트 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 매개 존재 검증 리스트 : ', len(card_weekend_list), card_weekend_list)
print('편의점 평일 매개 존재 검증 리스트 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 매개 존재 검증 리스트 : ', len(gs_weekend_list), gs_weekend_list)

카드 평일 매개 존재 검증 리스트 :  69 ['고산1동', '고산2동', '고산3동', '두산동', '만촌1동', '만촌2동', '만촌3동', '범물1동', '범물2동', '범어1동', '범어2동', '범어3동', '범어4동', '상동', '수성1가동', '수성2.3가동', '수성4가동', '중동', '지산1동', '지산2동', '파동', '황금1동', '황금2동', '남산1동', '남산2동', '남산3동', '남산4동', '대봉1동', '대봉2동', '대신동', '동인동', '삼덕동', '성내1동', '성내2동', '성내3동', '공릉1동', '공릉2동', '상계10동', '상계1동', '상계2동', '상계3.4동', '상계5동', '상계6.7동', '상계8동', '상계9동', '월계1동', '월계2동', '월계3동', '중계1동', '중계2.3동', '중계4동', '중계본동', '하계1동', '하계2동', '광희동', '다산동', '동화동', '명동', '소공동', '신당5동', '신당동', '약수동', '을지로동', '장충동', '중림동', '청구동', '필동', '황학동', '회현동']
카드 휴일 매개 존재 검증 리스트 :  69 ['고산1동', '고산2동', '고산3동', '두산동', '만촌1동', '만촌2동', '만촌3동', '범물1동', '범물2동', '범어1동', '범어2동', '범어3동', '범어4동', '상동', '수성1가동', '수성2.3가동', '수성4가동', '중동', '지산1동', '지산2동', '파동', '황금1동', '황금2동', '남산1동', '남산2동', '남산3동', '남산4동', '대봉1동', '대봉2동', '대신동', '동인동', '삼덕동', '성내1동', '성내2동', '성내3동', '공릉1동', '공릉2동', '상계10동', '상계1동', '상계2동', '상계3.4동', '상계5동', '상계6.7동', '상계8동', '상계9동', '월계1동', '월계2동', '월계3동', '중계1동', '중계2.3동', '중계4동',

In [ ]:
# 부트스트래핑 기각 제거 & 직접효과 제거 & a 음수 c양수 아닌 것들 제거
# 카드
card_weekday_list = set(card_weekday_list).difference(set(card_direct_mediation_weekday)).difference(set(list(card_boot_weekday[card_boot_weekday['ac']<=0].HDONG_NM)))
card_weekend_list = set(card_weekend_list).difference(set(card_direct_mediation_weekend)).difference(set(list(card_boot_weekend[card_boot_weekend['ac']<=0].HDONG_NM)))

print('카드 평일 부트스트랩, 직접효과 제거 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 부트스트랩, 직접효과 제거 : ', len(card_weekend_list), card_weekend_list)

# 편의점 
gs_weekday_list = set(gs_weekday_list).difference(set(gs_direct_mediation_weekday)).difference(set(list(gs_boot_weekday[gs_boot_weekday['ac']<=0].HDONG_NM)))
gs_weekend_list = set(gs_weekend_list).difference(set(gs_direct_mediation_weekend)).difference(set(list(gs_boot_weekend[gs_boot_weekend['ac']<=0].HDONG_NM)))

print('편의점 평일 부트스트랩, 직접효과 제거 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 부트스트랩, 직접효과 제거 : ', len(gs_weekend_list), gs_weekend_list)

print('-----------------------------------------------')
# 부트스트래핑 기각 제거 & 직접효과 제거 & a 음수 c양수 아닌 것들 제거
# 카드
card_weekday_list = set(card_weekday_list).difference(set(list(card_weekday_plus_minus.index)))
card_weekend_list = set(card_weekend_list).difference(set(list(card_weekend_plus_minus.index)))

print('카드 평일 a음수 c양수 아닌 것 제거 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 a음수 c양수 아닌 것 제거 : ', len(card_weekend_list), card_weekend_list)

# 편의점 
gs_weekday_list = set(gs_weekday_list).difference(set(list(gs_weekday_plus_minus.index)))
gs_weekend_list = set(gs_weekend_list).difference(set(list(gs_weekend_plus_minus.index)))

print('편의점 평일 a음수 c양수 아닌 것 제거 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 a음수 c양수 아닌 것 제거 : ', len(gs_weekend_list), gs_weekend_list)



카드 평일 부트스트랩, 직접효과 제거 :  50 {'상계10동', '수성4가동', '범어3동', '대봉2동', '중계4동', '상계9동', '수성1가동', '삼덕동', '동화동', '회현동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '수성2.3가동', '대신동', '만촌3동', '월계1동', '범어4동', '명동', '상계1동', '하계2동', '남산2동', '지산2동', '범어1동', '월계2동', '황학동', '상계5동', '남산1동', '신당5동', '중계2.3동', '소공동', '남산3동', '범물1동', '남산4동', '상동', '두산동', '공릉2동', '하계1동', '중림동', '성내2동', '파동', '중계1동', '공릉1동', '을지로동', '성내1동', '상계3.4동', '청구동', '만촌1동'}
카드 휴일 부트스트랩, 직접효과 제거 :  49 {'상계10동', '대봉2동', '중계4동', '상계6.7동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '고산1동', '대신동', '만촌3동', '명동', '범어4동', '상계1동', '성내3동', '남산2동', '월계2동', '범어1동', '다산동', '신당동', '상계5동', '고산2동', '남산1동', '상계8동', '중계2.3동', '신당5동', '필동', '소공동', '범물1동', '남산4동', '상계2동', '상동', '두산동', '동인동', '공릉2동', '중계본동', '황금1동', '하계1동', '고산3동', '중림동', '성내2동', '중동', '파동', '약수동', '범물2동', '을지로동', '상계3.4동', '만촌1동'}
편의점 평일 부트스트랩, 직접효과 제거 :  45 {'상계10동', '수성4가동', '범어3동', '상계9동', '수성1가동', '삼덕동', '상계6.7동', '회현동', '장충동', '황금2동', '범어2동', '고산1동', '만촌3동', '월계1동', '범어4동', '상계1동', '성내3동', '범어

## 최종 상관 분석 진행할 행정동 결과 csv 저장
- 결과 csv : boot_direct_mediation_result.csv

In [ ]:
# 부트스트랩 & 직접효과, 부호 ac  제거 결과 저장
boot_direct_mediation_result = [','.join(list(card_weekday_list)),
                    ','.join(list(card_weekend_list)),
                    ','.join(list(gs_weekday_list)),
                    ','.join(list(gs_weekend_list))]

boot_direct_mediation_result = pd.DataFrame(columns=['boot_direct_mediation_result'], data=boot_direct_mediation_result)
boot_direct_mediation_result.to_csv('/content/drive/My Drive/big_contest/bootstraping/boot_direct_mediation_result.csv', index=False, encoding='cp949')

print('부트스트랩, 직접효과, 부호 ac 제거 결과 csv 저장 완료')

부트스트랩, 직접효과, 부호 ac 제거 결과 csv 저장 완료


In [ ]:
# 부트스트랩 & 직접효과, 부호 ac 제거 결과 read 해보기

# 부트스트랩 & 직접효과, 부호 ac 제거 결과
boot_direct_mediation_result = pd.read_csv('/content/drive/My Drive/big_contest/bootstraping/boot_direct_mediation_result.csv', encoding='cp949')

# 부트스트랩 & 직접효과, 부호 ac 제거 결과
card_weekday_list = boot_direct_mediation_result.iloc[0, ].values[0].split(',')
card_weekend_list = boot_direct_mediation_result.iloc[1, ].values[0].split(',')
gs_weekday_list = boot_direct_mediation_result.iloc[2, ].values[0].split(',')
gs_weekend_list = boot_direct_mediation_result.iloc[3, ].values[0].split(',')

print('카드 평일 부트스트랩, 직접효과, 부호 ac 제거 : ', len(card_weekday_list), card_weekday_list)
print('카드 휴일 부트스트랩, 직접효과, 부호 ac 제거 : ', len(card_weekend_list), card_weekend_list)
print('편의점 평일 부트스트랩, 직접효과, 부호 ac 제거 : ', len(gs_weekday_list), gs_weekday_list)
print('편의점 휴일 부트스트랩, 직접효과, 부호 ac 제거 : ', len(gs_weekend_list), gs_weekend_list)

카드 평일 부트스트랩, 직접효과, 부호 ac 제거 :  44 ['상계10동', '수성4가동', '범어3동', '대봉2동', '중계4동', '수성1가동', '삼덕동', '회현동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '수성2.3가동', '대신동', '만촌3동', '월계1동', '범어4동', '명동', '하계2동', '남산2동', '범어1동', '월계2동', '상계5동', '남산1동', '신당5동', '중계2.3동', '소공동', '남산3동', '범물1동', '남산4동', '상동', '두산동', '공릉2동', '하계1동', '중림동', '성내2동', '중계1동', '공릉1동', '을지로동', '성내1동', '상계1동', '청구동', '만촌1동']
카드 휴일 부트스트랩, 직접효과, 부호 ac 제거 :  45 ['상계10동', '대봉2동', '중계4동', '상계6.7동', '장충동', '황금2동', '대봉1동', '만촌2동', '범어2동', '고산1동', '대신동', '만촌3동', '명동', '범어4동', '성내3동', '남산2동', '월계2동', '범어1동', '신당동', '상계5동', '고산2동', '남산1동', '중계2.3동', '신당5동', '필동', '소공동', '상계2동', '범물1동', '남산4동', '중계본동', '상동', '두산동', '동인동', '공릉2동', '황금1동', '하계1동', '고산3동', '중림동', '성내2동', '중동', '파동', '약수동', '을지로동', '상계1동', '만촌1동']
편의점 평일 부트스트랩, 직접효과, 부호 ac 제거 :  45 ['상계10동', '수성4가동', '범어3동', '상계9동', '수성1가동', '삼덕동', '상계6.7동', '회현동', '장충동', '황금2동', '범어2동', '고산1동', '만촌3동', '월계1동', '범어4동', '상계1동', '성내3동', '범어1동', '신당동', '황학동', '상계5동', '고산2동', '남산1동', '신당5동', '필동', '